In [45]:
import dash
from dash import dash_table
import pandas as pd
from dash import dcc
from dash import html
from dash import Input, Output, callback


# load the dataset.

url = 'https://raw.githubusercontent.com/liaochris/SPEOC-pt-1/main/data_clean/final_data_CD.csv'

df = pd.read_csv(url) 
df = df.drop(['Group Match Index', 'Group Match Url', 'Full Search Name', 'assets', 'Name_Fix_Transfer', 'Name_Fix_Clean', 'imputed_location', 'location conflict', 'Group Village'], axis=1)

df = df.rename(columns={'Unnamed: 0': 'Order', '6p_total': 'Face Value of 6% debt', '6p_def_total': 'Face Value of deferred 6% debt', 'unpaid_interest': 'Unpaid Interest', 'final_total': 'Final Total'})

df[['Face Value of 6% debt', 'Face Value of deferred 6% debt', 'Unpaid Interest', '6p_total_adj', '6p_def_total_adj', 'unpaid_interest_adj', 'Final Total', 'final_total_adj']] = df[['Face Value of 6% debt', 'Face Value of deferred 6% debt', 'Unpaid Interest', '6p_total_adj', '6p_def_total_adj', 'unpaid_interest_adj', 'Final Total', 'final_total_adj']].round(0)

# Initialize the app
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("DataTable of Creditors", style={'text-align': 'center'}), # name the Main Headline H1
   
    dash_table.DataTable(
        id = 'DataTable',
        data=df.to_dict('records'),  # convert the pd dataframe into a dictionary, otherwise Dash cannot process it.
        columns=[{"name": i, "id": i} for i in df.columns],
        # first two arguments of dash_table.DataTable are data & columns by default.
        
        style_cell={'textAlign': 'left'}, # align the texts in columns to the left
        style_cell_conditional=[
        {
            'if': {'column_id': 'Region'},
            'textAlign': 'left'
        }
    ],
        
      
        style_header={
        'backgroundColor': 'grey',
        'fontWeight': 'bold'
    }, # To style the header, I changed the color of our header (random color) and make the font bold.
        
        style_data={
        'backgroundColor': 'white',
        'color': 'black'
    }, # I can also change the color of data.
        
# Interactivity starts here:
        editable=True,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",

        selected_columns=[],
        selected_rows=[],
        page_size= 30,

        
    ),
    
        dcc.Dropdown(
        id='dropdown',
        options=[
            {'label': 'Group by State', 'value': 'state'},
            {'label': 'Group by Occupation', 'value': 'occupation'}
        ],
        value='state'
    ),
    
    html.Br(),
    html.Label("Please select the number of records you want to display:"),
    html.Div(
        id='state-slider-container',
        children=[
            dcc.Slider(
                id='state-slider',
                min=10,
                max=df['Group State'].nunique(),
                step=1,
                value=10,
                marks={10: '10', df['Group State'].nunique(): str(df['Group State'].nunique())},
            )
        ]
    ),

    html.Div(
        id='occupation-slider-container',
        children=[
            dcc.Slider(
                id='occupation-slider',
                min=10,
                max=df['occupation'].nunique(),
                step=1,
                value=10,
                marks={10: '10', df['occupation'].nunique(): str(df['occupation'].nunique())},
            )
        ]
    ),
    
    html.Div(id = 'DataTable Container')
])


@callback(
    Output('DataTable Container', "children"),
    [Input('DataTable', "derived_virtual_data"),
    Input('DataTable', "derived_virtual_selected_rows"),
    Input('dropdown', "value"),
    Input('state-slider', 'value'),
    Input('occupation-slider', 'value')])

def update_graphs(rows, derived_virtual_selected_rows, dropdown_value, state_n, occupation_n):
    
    if derived_virtual_selected_rows is None:
        derived_virtual_selected_rows = []

    dff = df if rows is None else pd.DataFrame(rows)
    
    # aggregate data
    agg_columns = ["Face Value of 6% debt", "Face Value of deferred 6% debt", "Unpaid Interest", "Final Total"]
    dff_group_state = dff.groupby("Group State")[agg_columns].sum().reset_index()
    
    # load the dataset.
    df_occupation = dff 

    # Split "Occupation" column on "|" and explode it into multiple rows
    df_occupation['occupation'] = df_occupation['occupation'].str.split('|')
    df_occupation = df_occupation.explode('occupation')
    df_occupation['occupation'] = df_occupation['occupation'].str.strip()  # remove leading and trailing spaces
    dff_occupation = df_occupation.groupby("occupation")[agg_columns].sum().reset_index()
    
    
    colors = ['#7FDBFF' if i in derived_virtual_selected_rows else '#0074D9'
              for i in range(len(dff))]
    if dropdown_value=='state':
        charts = [
            dcc.Graph(
                id=column,
                figure={
                    "data": [
                        {
                            "x": dff_group_state.sort_values(by=column, ascending=False)["Group State"].head(state_n),
                            "y": dff_group_state.sort_values(by=column, ascending=False)[column].head(state_n),
                            "type": "bar",
                            "marker": {"color": colors},
                        }
                    ],
                    "layout": {
                        "xaxis": {"automargin": True},
                        "yaxis": {
                            "automargin": True,
                            "title": {"text": column}
                        },
                        "height": 250,
                        "margin": {"t": 10, "l": 10, "r": 10},
                    },
                },
            )
            for column in ["Face Value of 6% debt", "Face Value of deferred 6% debt", "Unpaid Interest", "Final Total"] if column in dff
        ]
    else: 
        
        charts = [
            dcc.Graph(
                id=column+'-occupation',
                figure={
                    "data": [
                        {

                            "x": dff_occupation.sort_values(by=column, ascending=False)["occupation"].head(occupation_n),
                            "y": dff_occupation.sort_values(by=column, ascending=False)[column].head(occupation_n),
                            "type": "bar",
                            "marker": {"color": colors},
                        }
                    ],
                    "layout": {
                        "xaxis": {"automargin": True},
                        "yaxis": {
                            "automargin": True,
                            "title": {"text": column}
                        },
                        "height": 250,
                        "margin": {"t": 10, "l": 10, "r": 10},
                    },
                },
            )
            for column in ["Face Value of 6% debt", "Face Value of deferred 6% debt", "Unpaid Interest", "Final Total"] if column in dff
        ]
        
        

    return charts

@app.callback(
    [Output('state-slider-container', 'style'),
     Output('occupation-slider-container', 'style')],
    [Input('dropdown', 'value')]
)
def toggle_slider(dropdown_value):
    if dropdown_value == 'state':
        return {'display': 'block'}, {'display': 'none'}
    elif dropdown_value == 'occupation':
        return {'display': 'none'}, {'display': 'block'}


if __name__ == '__main__':
    app.run_server(debug=True, port=8062)

In [42]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3893 entries, 0 to 3892
Data columns (total 15 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Order                           3893 non-null   int64  
 1   Group Name                      3893 non-null   object 
 2   Group State                     3868 non-null   object 
 3   Group County                    3109 non-null   object 
 4   Group Town                      2704 non-null   object 
 5   Group Name Type                 3893 non-null   object 
 6   occupation                      0 non-null      float64
 7   Face Value of 6% debt           3893 non-null   float64
 8   Face Value of deferred 6% debt  3893 non-null   float64
 9   Unpaid Interest                 3893 non-null   float64
 10  6p_total_adj                    3893 non-null   float64
 11  6p_def_total_adj                3893 non-null   float64
 12  unpaid_interest_adj             38